# Introducción

El campo de la Teoría de Grafos ha sido ampliamente utilizado en los últimos años para modelizar y analizar datos biológicos de diversa naturaleza, como redes de interacción de proteínas o redes metabólicas. Esto se debe a la versatilidad que ofrece este tipo de representaciones ya que, además de otro tipo de análisis, permiten conocer su topología, la detección de sesgos estructurales o la identificación de nodos más relevantes dentro de la red, características que ayudan a la comprensión de los sistemas biológicos. En concreto, en el presente estudio se ha llevado a cabo el análisis de la red de interacción de proteínas de *Drosophila melanogaster* (mosca de la fruta), organismo modelo de gran importancia que ha sido y es utilizado para entender la biología, el desarrollo y las enfermedades humanas. Los datos utilizados han sido obtenidos del apartado de datasets de ejemplo de la base de datos GraphWeb, pero proceden de la base de datos sobre redes de interacción de proteínas del EMBL-EBI (IntAct). 


# Métodos

El análisis y la visualización de la red han sido llevados a cabo mediante el módulo de Python ``nerworkx`` (NetworkX 2.4) y el software Cytoscape (Cytoscape 3.7.2). Respecto a la búsqueda de comunidades, se ha utilizado el módulo ``community`` (Community 0.13), que implementa el algoritmo de Louvain. Los análisis de enriquecimiento de genes se han llevado a cabo mediante el paquete ``clusterProfiler`` (clusterProfiler 3.14.3) de Bioconductor a través de R. 


# Análisis (serían los resultados, lo llamamos como tú quieras)

## Descripción.

La red proteica de estudio está contenida en un fichero *.txt* en formato de lista de adyacencia. Cada una de las proteínas está representada por un nodo, que se une a otros mediante ramas, solo en el caso de que existan evidencias experimentales de interacción entre ellas (**ME LO ESTOY INVENTANDO**). El análisis preliminar de la red permite saber que presenta un orden de 8080 nodos y un tamaño de 26197 ramas. Además, dicha red no presenta ponderación ni dirección (grafo no ponderado y no dirigido). Por otro lado, el análisis de su densidad revela un coeficiente de 0.0008026, por lo que no se puede considerar que sea una red densa (**mirar significado coeficiente de densidad**). La densidad de una red es su número de ramas / el número máximo de ramas que puede tener (un grafo no dirigido puede tener como máximo $\frac{N \cdot (N – 1)}{2}$ ramas, siendo N el número de nodos). Esto implica que la densidad sea un número entre 0 y 1 de forma que, en el caso de que la densidad sea cercana a 1, se tratará de un grafo denso mientras que, en el caso de que sea cercana a 0, será disperso. Esto justifica la conclusión de que el grafo de estudio sea disperso, pues es un número muy cercano a 0.

El análisis de redes mediante grafos permite detectar funcionalidades de proteínas. Para ello, se suele calcular el número Q, número de veces que aparece un fenómeno cuando se analizan miles de grafos aleatorios con el mismo número de nodos y ramas. Si el número Q es bajo, significa que hay un sesgo estructural en la red y es algo a lo que hay que presentar atención. Si el número Q es alto, quiere decir que el fenómeno es algo normal en grafos de mismo orden y tamaño, y no es interesante. Al explorar algo, hay que demostrar así que representa un sesgo estructural en la red de interés, es atípico. Dado el alto número de nodos de la red de estudio y las limitaciones computacionales, se va a proceder a comparar el grafo de estudio con un grafo aleatorio equivalente (mismo número de nodos y ramas), para lo que se van a analizar una serie de métricas:

* Conectividad y número de componentes conexas: Un grafo es conexo si para cualquier par de nodos hay un camino que los une. Cuando un grafo no es conexo, sin embargo, se pueden buscar sus componentes conexas: subgrafo maximal (no se pueden añadir más nodos) de forma que es conexo. Al tratarse la red de interés de un grafo disconexo, se puede dividir en 69 componentes conexas, de las cuales cabe destacar que aquella con mayor tamaño representa el *core* del grafo y, en este caso, constituye prácticamente la totalidad de la red, ya que está formada por 7942 nodos y 26116 ramas, es decir, el 98.29% del total de nodos y el 99.69% del total de ramas. El resto de componentes conexas consisten principalmente en nodos autoconectados y motivos de dos nodos conectados entre sí. El grafo aleatorio equivalente también es disconexo, pero solo presenta 10 componentes conexas, aunque aquella de mayor tamaño también constituye la práctica totalidad de la red (8071 nodos y 26197 ramas).


* Grado: El grado de un nodo es el número de vecinos del mismo. Así, se puede construir un diccionario de Python donde cada elemento sea cada nodo del grafo y su valor el grado. Esto permite ver que el nodo CG12470 es el de mayor grado del grafo original, siendo éste 176, mientras que el del grafo aleatorio equivalente tiene un grado de 20 (1727-ésimo).


* Betweenness: Dado un nodo $v_i$, se define su betweenness como la fracción de caminos mínimos que hay entre el resto de nodos del grafo y que pasan por dicho nodo. Esto se hace para todos los nodos del grafo. Cabe destacar que un paseo es una secuencia de vértices con la condición de que, para pasar de uno a otro, debe haber una rama
que los conecte; un paseo en el cual todos los vértices son distintos, se denomina camino; y el el camino mínimo es el camino más corto de un nodo a otro. Solo saber si hay más de dos caminos mínimos entre dos nodos es un problema NP-completo, de forma que calcular el betweenness es complicado, pero se pueden usar aproximaciones. Una vez calculado betweenness de cada nodo, nos interesan los nodos con betweeness alto porque significa que la mayoría de caminos mínimos pasan por ese nodo y, atacándolo, se afectaría a la red entera. Además, dicho cálculo permite calcular el betweenness de un grafo como el promedio del de todos los nodos del mismo (número entre 0 y 1). La comparación del betweenness entre el grafo original y el aleatorio equivalente permite ver que el nodo con mayor betweenness tiene un valor de 0.0397 (CG12470) y 0.0036 (2161-ésimo), respectivamente. La comparación permite determinar que el betweenness de CG12470 es alto, habiendo un sesgo que determina que la mayoría de caminos mínimos pasen por ese nodo.


* Closeness: La cercanía o closeness de un nodo es la inversa de la suma de las distancias de un nodo a todos los demás del grafo. Un closeness alto significa así que todos los nodos están cerca unos de otros. La comparación del closeness entre el grafo original y el aleatorio equivalente permite ver que el nodo con mayor closeness tiene un valor de 0.3256 (CG12470) y 0.2324 (305-ésimo), respectivamente. Así, se puede ver que el nodo de mayor grado, betweennes y closeness del grafo original es CG12470, mientras que cambia en cada caso para el aleatorio.


* Diámetro: El diámetro de un grafo es la máxima distancia entre cualquier par de nodos. Sin embargo, el cálculo de esta métricasolo es aplicable a grafos conexos de forma que, en el caso del grafo original y del aleatorio equivalente, solo se puede implementar para la componente conexa de mayor tamaño <font color='red'>(EN EJECUCIÓN)</font>.


* Máximo k para el cual existe un k-core: Un k-core es un subconjunto de nodos tal que en todos ellos el grado es k o más. El máximo k para el cual existe un k-core es de 11 para el grafo original y de 4 para el aleatorio equivalente. Cabe destacar que este cálculo se lleva a cabo excluyendo los posibles bucles que pueda tener el grafo.


* Índice de clusterización: El índice de clusterización es la probabilidad local de que dos vecinos de un nodo dado estén conectados entre sí. El índice de clusterización de un grafo se puede así calcular como el promedio del de todos los nodos del mismo. El índice de clusterización promedio de la red de estudio es 0.02168 y el de la red aleatoria equivalente es de 0.00062.


* Camino característico: El camino característico indica en promedio cómo de cerca están unos nodos de otros. Así, para calcular el camino característico de un grafo, se calcula el de cada nodo (la distancia de ese nodo a todos los demás, divido por el número de vértices menos 1) y, después, se calcula la media. Dado que ambos grafos estudiados no son conexos, su camino característico promedio se puede denotar como el número de nodos - 1 (8079) o se puede calcular el de la componente conexa de mayor tamaño de dicho grafo, cuyos valores son de 4.371 y 5.0247 para el grafo original y el aleatorio, respectivamente.


* Lazos: Un lazo es una rama que empieza y acaba en el mismo nodo. En el caso del grafo de interés, hay 251 lazos. En el caso del aleatorio, no hay lazos.


* Ciclos: Un paseo cerrado es un paseo tal que los nodos inicial y final son el mismo. Un paseo cerrado en el que no se repiten ramas es un circuito. Un ciclo es un circuito en el que no se repiten vértices. Es importante encontrar ciclos porque los grafos aleatorios no suelen tenerlos. Pues en este caso hay 149 ciclos en el aleatorio frente a los 56 del original.


* Cliqué: Un cliqué es un subgrafo completo, es decir, un subgrafo donde cada nodo está conectado con todos los demás. En el grafo original, el número de cliqués es de 38048, frente a los 34321 del aleatorio. El tamaño del cliqué más grande de cada uno es de 8 y 3, respectivamente.


A continuación, se muestra una tabla con el resumen de la comparación de las métricas más importantes obtenidas durante el análisis de la red:

Tabla con métricas. 

Respecto a la distribución de grados de la red (Figura 1), se puede observar la presencia de una gran cantidad de nodos con grado entre 0 y 25 (7641 nodos, 95% de la red), mientras que el número desciende enormemente conforme aumenta el número de vecinos, habiendo solo 25 nodos con un grado mayor o igual que 75 y 9 con un grado mayor o igual que 100. Esta distribución es típica de las redes libres de escala, caracterizadas por la presencia de agrupaciones de nodos, llamados *hubs*, que muestran una gran cantidad de conexiones en relación al resto de la red, que se mantiene con una densidad de conexión relativamente baja.

Además, las redes libres de escala se caracterizan porque la distribución de grado de sus nodos en escala logarítmica-logarítmica se ajusta a una recta de pendiente negativa que al final tiene un corte o cutoff. Este comportamiento de la distribución de grado se diferencia de lo que ocurre en un grafo aleatorio (sigue una Poisson) o en uno regular (sigue una Delta) (Figura 2).

Para determinar si verdaderamente se trata de una red de este tipo, es necesario hacer uso de la comparación con grafos aleatorios.


## Sesgos estructurales: comparación con redes aleatorias.

Con el fin de encontrar algún tipo de sesgo estructural, se procede a la comparación de las características más importantes del grafo de interés con las del grafo aleatorio del mismo tamaño y orden (generado mediante la función ``gnm_random_graph``). Como se puede observar en el apartado anterior, tanto durante el detallado de cada una de las métricas como en la tabla resumen, prácticamente todas las métricas apoyan la existencia de un sesgo estructural en la red de interés de *D. melanogaster*.

Empezando por la conectividad, ninguno de los grafos es conexo, pero el número de componentes conexas es de 69 frente a las 10 del grafo aleatorio equivalente. ¿¿Iba a hablar de la existencia de grupos que pueden llevar a cabo funciones parecidas, o formados por proteínas que llevan a cabo la misma función, pero he caído en que la componente conexa de mayor tamaño es prácticamente toda la red de interacción. A ver, eso deja entrever que existe una conexión necesaria para que todo el orgnismo funcione de forma regulada, pero no sé muy bien qué redactar al respecto??

El sesgo se evidencia mucho más con el análisis del grado (valor del nodo con grado más alto, distribución de los grados...), como se ha explicado en el anterior apartado. También ocurre al observar métricas como el betweenness y el closeness. Dadas sus definiciones, el sesgo se manifiesta sobre todo por el valor máximo en cada grafo. En el grafo de interés, el nodo CG12470 es el de mayor grado, betweenness y closeness, características que le llevan a ser uno de esos *hubs* típicos de las redes libres de escala. Además, como se puede observar en el apartado anterior, el nodo con mayor betweenness y el de mayor closeness no coinciden en la red aleatoria, y sus valores son menores que los de CG12470.

También es importante destacar métricas como el índice de clusterización y camino característico. Referente a ellas, las propiedades de las redes libres de escala suelen ser: camino característico corto, incluso más que en el grafo aleatorio equivalente; índice de clusterización pequeño y decreciente (a mayor número de nodos), pero mayor que el del grafo aleatorio equivalente. En este caso, el índice de clusterización de la red de estudio es varios órdenes de magnitud mayor que el de la red aleatoria equivalente, mientras que su camino característico es más pequeño que el de la aleatoria. Así, ambas propiedades se cumplen.

Por último, llama la atención el máximo k para el cual existe un k-core. Esta métrica también revela la existencia de un sesgo determinado en la red de interés pues, en ella, dicho valor es de 11, mientras que el de la red aleatoria equivalente es 4. Además, si se construyen 100 grafos aleatorios equivalentes y se mira en cuantos de ellos se obtiene un k máximo mayor o igual que 11, no se obtiene ninguno. Por tanto, en la red de *D. melanogaster* existe un grupo de proteínas en el que cada una tiene 11 vecinos, las cuales deben tener un papel importante dentro de la red.

Comparaciones de métricas rollo número de bucles, k-cores, ...

Comparación de medidas de centralidad: se puede meter, además de una aleatoria, una libre de escala. 

Comparación de distribución de grados de libre de escala vs aleatoria.


## Ataques a redes

Un ataque a la red es una serie de elementos de la misma que se quitan. El objetivo de un ataque es producir el mayor daño posible en una red y el objetivo de una red es que un ataque le produzca el menor daño posible. Se pueden llevar a cabo distintas estrategias que, en función del tipo de grafo sobre el que se ejecuten, tendrán más o menos relevancia. Entre ellas, los ataques pueden dirigirse a eliminar nodos o ramas con ciertas características.

Cuánto resiste un grafo a un ataque se puede medir de muchas maneras: la conectividad (resistencia) de un grafo a un ataque se mide por cuántos nodos se han quedado desconectados de todos y cada uno de un conjunto finito de nodos. Otra forma es pensar que no hay nodos especiales y medir el daño de un ataque en términos de conectividad global: el ataque hace daño si la componente conexa más grande es pequeña y viceversa. La eficiencia (daño) de un algoritmo de ataque es inversamente proporcional a la resistencia del grafo a dicho ataque.

Dadas estas definciones/explicaciones, los grafos más resistentes son los grafos aleatorios porque no tienen un sesgo estructural del que tirar. Por tanto, se pueden definir distintas estrategias de ataque y observar el comportamiento de cada tipo de grafo frente a ellas.

Por ejemplo, los ataques aleatorios eliminan nodos de forma aleatoria e iterativa (un cierto número en cada iteración). Dada la topología libre de escala, existen muchos nodos con grado bajo y unos pocos con grado muy alto (*hubs*), de forma que existe una alta probabilidad de que una iteración no afecte en gran medida a la red pero, en caso de afectar a uno de los *hubs*, ésta se vería seriamente afectada. Sin embargo, esto no ocurre en el caso de las redes aleatorias, donde todos los nodos tienen un grado bajo, lo que las hace más resistentes a este tipo de ataques.

Otra estrategia sería ir atacando los nodos de mayor grado. Atendiendo a todas las explicaciones dadas hasta este punto, resulta obvio que este ataque afectará en mayor medida al grafo libre de escala que al aleatorio, aunque éste último no es tan resistente como en la anterior estrategia porque se eliminan de forma específica esos nodos de grado máximo.


## Búsqueda de comunidades

Bajo la premisa de que aquellas proteínas que presenten un mayor número de interacciones entre sí tendrán funciones similares o estarán involucradas en procesos biológicos relacionados, se lleva a cabo una aproximación típica del estudio de las redes sociales que en los últimos años ha cobrado mayor importancia en el campo de la Biología: la búsqueda de comunidades. Una comunidad consiste en una agrupación de nodos más densamente conectados entre sí que con otros nodos, de forma que, en el contexto de las redes de interacción de proteínas, representarán grupos de éstas que, en principio, estarán más relacionadas entre sí. Hay diferentes métodos para la detección de estas divisiones (métodos aglomerativos, etc.), pero los más populares se basan en la maximización de una función objetivo, la modularidad ($Q$). Se trata de una medida de la calidad de la división particular en comunidades de una red, y consise en un valor escalar entre -1 y 1 que mide la densidad de uniones dentro de las agrupaciones en comparación con la densidad de enlaces entre ellos. Esta métrica viene dada por la siguiente expresión: 

\begin{equation*}
    Q = \frac{1}{2m} \sum_{i,j}\left[A_{i,j} - \frac{k_i k_j}{2m}\right]\delta(c_i,c_j)  
\end{equation*}

donde $A_{ij}$ es el número de ejes existentes entre los nodos $i$ y $j$ (en este caso, será 0 o 1), $k_i$ y $k_j$ el grado de los nodos considerados, $\delta(c_i, c_j)$ una función Delta de Kronecker que solo es 1 en el caso de que los dos nodos pertenezcan a la misma red y $m$ el total de ejes de la red, dado por la expresión $\frac{1}{2} \sum_ik_1$. En definitiva, su cálculo consiste en el recorrido de todos los pares de nodos que componen la red y, solo para aquellos que pertenecen a la misma comunidad, se lleva a cabo el sumatorio de la diferencia entre el número de ejes presentes entre esos dos nodos (1 ó 0) menos el número de ejes que esperaríamos en una red aleatoria ($k_ik_j/2m$). Esta expresión es equivalente a la siguiente más intuitiva:  

\begin{equation*}
    Q = \sum_{i = 1}^c(e_{ii} - a^2_i)
\end{equation*}

en la que $c$ son las cumunidades de la red, $e_{ii}$ es la fracción de ejes cuyos nodos pertenecen a la comunidad $c$ y $a_i$ es la fracción de ejes que aleatoriamente pertenecen a la comunidad $c$. **meter definiciones**

Sin embargo, a pesar de que esta función permita en teoría obtener el óptimo global de $Q$ para una determinada red y, por definición, su partición óptima en comunidades, la cantidad de cálculos necesarios convierten al proceso en una tarea computacionalmente inviable, ya que se trata de un problema de optimización combinatorial NP-completo. Para reducir el número de veces que $Q$ es calculada, existen diferentes métodos heurísticos que buscan hacer converger la red en agrupaciones con cada vez un mayor número de nodos en función de $Q$, siendo el método de Louvain la aproximación utilizada en el presente estudio. El algoritmo está compuesto por la repetición iterativa de dos fases que, a través de la partición jerárquica de la red, se reproducirán hasta que $Q$ converga a un máximo local.

En el presente análisis, tal y como se ha comentado en la sección de Métodos, se ha utilizado la implementación del método de Louvain del módulo ``community`` de Python debido a su alta integración con el módulo ``networkx``. Dado que se trata de un algoritmo relativamente rápido a pesar del tamaño y orden de la red, se ha repetido el proceso de búsqueda 100 veces reteniendo aquella configuración con mayor modularidad. El proceso se llevó a cabo sobre la componente conexa de mayor tamaño (26.116 ramas y 7.942 nodos), ya que no tiene sentido la búsqueda de comunidades sobre un grafo disjunto, y sobre un grafo aleatorio del mismo tamaño y orden generado mediante la función ``gnm_random_graph`` con propósitos comparativos. 

Respecto a los resultados, en primer lugar cabe destacar que, tanto en el caso de la red aleatoria como en el de la red biológica, la modularidad resultante entre las 100 iteraciones llevadas a cabo fue muy similar, presentando desviaciones estándar muy bajas (Tabla 1), lo que indica que el algoritmo, a pesar de ser heurístico, converge hacia resultados similares. Debido a cómo está construido el algoritmo, el cual no se detiene hasta que no encuentra convergencia para los valores de $Q$ mediante la heurística que sigue, se puede obtener información relativa a la modularidad que presenta cada red. Mientras que en el caso de la red biológica el tiempo de ejecución por iteración fueron 3min 20s ± 3.37s (media ± desviación estándar de 7 ejecuciones), en el caso de la red aleatoria aumentó a 8min 32s ± 22.2s (tiempos medidos mediante el módulo ``timeit`` de Python). Esto significa que el algoritmo tardó en encontrar convergencia hasta más de dos veces más en la red aleatoria respecto a la biológica, lo que a priori denota diferencias en sus topologías, ya que ambas presentan el mismo orden y tamaño. 

Respecto a la comparativa entre la red de estudio y la red aleatoria, se puede observar que la primera presenta una modularidad más alta que la segunda tanto en el máximo obtenido como en el valor medio de las 100 iteraciones llevadas a cabo. Respecto al número de comunidades detectadas, la red de estudio presenta un menor número que la red aleatoria (Tabla 1). Esto se debe a que la red biológica presenta un sesgo estructural hacia la presencia de agrupaciones de nodos más densamente conectados entre sí que con otros, tanto por el hecho de presentar características similares a las redes libres de escala (mirar sección **X**) como por el hecho de ser proteínas con funciones relacionadas. En la red aleatoria, en cambio, no se espera ese tipo de topología, lo que explica su modularidad más baja y un mayor número de comunidades. 

También es relevante el estudio de la topología que presentan las comunidades obtenidas en términos de número de nodos. En la Tabla 2 se puede observar que la red de Drosophila presenta comunidades con un orden medio mayor que la red aleatoria, llamando la atención la presencia de varias comunidades de gran tamaño que podrían corresponder a grandes redes de interacción (Figura **X**). En el caso de la red aleatoria, el orde de las comunidades obtenidas es menor al obtenido en la red de estudio tanto respecto a la media como a la comunidad con mayor número de nodos. Todo ello, sumado a las diferencias presentadas en $Q$, parece indicarnos un sesgo estructural de nuestra red en términos de modularidad.


Para una interpretación más visual, se llevó a cabo representación gráfica de ambas redes agrupando los nodos que pertenecen a una determinada comunidad mediante el algoritmo de *layout* Compound Spring Embedder (CoSE) que ofrece la app AutoAnnotate de Cytoscape. Tras su ejecución, para evitar el solapamiento de los grupos y hacer más legible el gráfico, se han separado manualmente hacia los extremos las comunidades con un orden mayor o igual que 300, posicionando el resto en el centro (Figura **X**). Llama la atención cómo se distribuyen los nodos con mayor grado en las comunidades de las dos redes. Mientras que en el caso de la red aleatoria hay una distribución uniforme producto de presentar una distribución de grados con forma de Poisson (Figura **X**), en el caso de la red de Drosophila podemos observar que los nodos con mayor grado se localizan en el interior de las comunidades, sobre todo en aquellas que presentan un mayor número de nodos. Estos resultados indican que las comunidades obtenidas en este caso son mucho más robustas que en el caso de la red aleatoria, ya que los nodos con un mayor número de conexiones se localizan en el núcleo, mientras que aquellos con menor grado unen unas comunidades con otras, lo que refuerza la idea de que presenta una estructura modular. Esta topología no es posible observarla en la red aleatoria, ya que no presentan una unión preferencial entre los nodos, sino que éstos se unen con otros de forma aleatoria. 


Además del análisis topológico de las comunidades obtenidas, se ha llevado a cabo un estudio funcional de las proteínas que las conforman mediante un análisis de enriquecimiento de genes basado en términos GO (Gene Ontology) de *Drosophila*  con el fin de determinar si los resultados tienen algún tipo de interpretación biológica. Para ello, se ha utilizado el paquete ``clusterProfiler`` de R, utilizando como *query* aquellas comunidades con un orden mayor o igual a 600 nodos (5 comunidades) con el fin de obtener mayor significancia estadística y por el hecho de que se tratan de las agrupaciones que a priori parecen más robustas. Las entradas en la base de datos GO representan, dentro de los 3 tipos de ontologías (Proceso Biológico (PB), Función Molecular (FM) y Componente Celular (CC)), grupos de genes que tienen que ver con las funciones que describen, de forma que los resultados serán aquellas entradas en las que los grupos de genes introducidos como query se encuentran sobre-enriquecidos sobre el background genético total del organismo de estudio. Sobre las entradas obtenidas se ha llevado a cabo corrección por testeo múltiple de los p-valores asociados mediante el método de Benjamini-Hochberg (FDR) y se establece un umbral de significancia $\leq 0.05$. 

Los resultados indican que cada comunidad presenta proteínas relacionadas con una tarea específica. A continuación se comentarán las dos comunidades de mayor tamaño como ejemplo, pero se han encontrado resultados significativos para todas ellas. Respecto a la comunidad de mayor orden (1185 nodos), los Procesos Biológicos encontrados están relacionados con la regulación del ciclo celular y de la divisón celular a través de la organización de los microtúbulos que conforman el huso mitótico (Figura **X**). Respecto a las Funciones Moleculares, indican que muchas de las proteínas de este grupo están relacionadas con la unión al ADN, lo que indica que está formada principalmente por proteínas relacionadas con la replicación y la transcripción. La segunda comunidad con mayor orden (900 nodos), en cambio, está constituida por proteínas relacionadas con los procesos de transcripción-traducción, ya que la mayoría de ellas intervienen en el procesamiento de los mRNA. De hecho, observando los resultados de Componentes Celulares (Figura XB), las entradas con mayor ratio de genes y significancia corresponden con el complejo spliceosomal, por lo que parece que se tratan de proteínas que intervienen en el proceso de splicing durante la transcripción. 

Estos resultados demuestran la utilidad del uso de la Teoría de  Grafos para la representación de datos biológicos, ya que permite organizar la información en estructuras ricas en información sobre las que se pueden implementar aproximaciones como la comentada en el presente estudio, la búsqueda de comunidades, donde únicamente a partir de la información relativa a qué proteínas interaccionan entre sí, se han generado clústeres que biológicamente parecen guardar sentido. Posibles análisis posteriores podrían consistir en llevar a cabo la búsqueda de "metacomunidades" dentro de las obtenidas con el fin de buscar funciones más específicas que, en última instancia, podrían servir para la caracterización de proteínas cuya función no ha sido descrita. 

**Links con info**

* https://mathoverflow.net/questions/212761/modularity-in-a-graph-definition-of-the-random-component
* https://www.cs.cmu.edu/~ckingsf/bioinfo-lectures/modularity.pdf